In [46]:
import pandas as pd
import joblib

In [8]:
interaction_df = pd.read_csv("data/user_item_interactions.csv")
interaction_df.head()

,item_id,user_id,timestamp,click,purchase
0,6198,36c0daa2-5aad-4b0c-81e3-8d276b223573,1633082142,1,0
1,190,c9a9b40a-90ce-42c7-8b02-1301ad4914b1,1630125341,1,0
2,7330,83969882-083c-4150-b9eb-19bb3197720d,1630143229,1,0
3,8978,83969882-083c-4150-b9eb-19bb3197720d,1630143685,1,0
4,9263,83969882-083c-4150-b9eb-19bb3197720d,1630143781,1,0


In [16]:
interaction = interaction_df.copy()
interaction['purchase'] = interaction['purchase'].apply(lambda x: 2 if x==1 else 0)
interaction['rating'] = interaction['purchase'] + interaction['click']
interaction.head()

,item_id,user_id,timestamp,click,purchase,rating
0,6198,36c0daa2-5aad-4b0c-81e3-8d276b223573,1633082142,1,0,1
1,190,c9a9b40a-90ce-42c7-8b02-1301ad4914b1,1630125341,1,0,1
2,7330,83969882-083c-4150-b9eb-19bb3197720d,1630143229,1,0,1
3,8978,83969882-083c-4150-b9eb-19bb3197720d,1630143685,1,0,1
4,9263,83969882-083c-4150-b9eb-19bb3197720d,1630143781,1,0,1


In [21]:
dataset = pd.DataFrame()
dataset['user'] = interaction['user_id']
dataset['item'] = interaction['item_id']
dataset['rating'] = interaction['rating']
dataset['timestamp'] = interaction['timestamp']
dataset.head()

,user,item,rating,timestamp
0,36c0daa2-5aad-4b0c-81e3-8d276b223573,6198,1,1633082142
1,c9a9b40a-90ce-42c7-8b02-1301ad4914b1,190,1,1630125341
2,83969882-083c-4150-b9eb-19bb3197720d,7330,1,1630143229
3,83969882-083c-4150-b9eb-19bb3197720d,8978,1,1630143685
4,83969882-083c-4150-b9eb-19bb3197720d,9263,1,1630143781


In [68]:
dataset.to_csv('data/user_item_interactions.csv', index=False)

### N-Recommended Products

In [47]:
from surprise import KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import SVD


reader = Reader(rating_scale=(1, 2))
data = Dataset.load_from_df(df=dataset[['user', 'item', 'rating']], reader=reader)

kf = KFold(n_splits=10)

svd = SVD()

for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.1775
RMSE: 0.1789
RMSE: 0.1742
RMSE: 0.1774
RMSE: 0.1766
RMSE: 0.1762
RMSE: 0.1763
RMSE: 0.1747
RMSE: 0.1759
RMSE: 0.1766


In [66]:
from collections import defaultdict
import json
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)
result = {}
for uid, user_ratings in top_n.items():
    result[uid] = [iid for (iid, _) in user_ratings]
with open("result.json", "w") as f:
    f.write(json.dumps(result))

In [51]:
joblib.dump(svd,'svd.joblib')

['svd.joblib']

### K-Similar Items

In [43]:
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [44]:
testset = trainset.build_testset()
predictions = knn.test(testset)
accuracy.rmse(predictions, verbose=True)

RMSE: 0.0465


0.046461337102403036

In [48]:
knn.get_neighbors(456, k=10)

[689, 735, 67, 3104, 365, 688, 2414, 3107, 6947, 1202]

In [93]:
joblib.dump(knn, 'knn.joblib')

['knn.joblib']

In [94]:
model = joblib.load('knn.joblib')
model.get_neighbors(456, k=10)

[689, 735, 67, 3104, 365, 688, 2414, 3107, 6947, 1202]

### Popular Recommended Products

In [92]:
popular_items = dataset.groupby(['item']).size().reset_index()
popular_items['count'] = popular_items[0]
popular_items.sort_values(by='count', ascending=False).head(10)
# popular_items[0]

,item,0,count
2511,2719,3409,3409
3188,3589,3081,3081
7610,8978,2867,2867
2845,3082,2612,2612
1451,1547,2075,2075
2509,2717,1842,1842
10290,13400,1569,1569
1188,1264,1489,1489
717,761,1467,1467
176,189,1458,1458


In [91]:
dataset[(dataset['item'] == 2719)]

,user,item,rating,timestamp
581,689e1169-d896-4523-8a97-c70649763e67,2719,1,1629074397
583,d7d13b02-7da2-475c-ba5b-698833c87ef9,2719,1,1629100051
642,689e1169-d896-4523-8a97-c70649763e67,2719,1,1629419326
1233,ef5a9bc2-fbcb-4466-a887-70dce4e7cf85,2719,1,1631925066
1295,11e464cf-31c0-4968-ae4d-188ee0aa8664,2719,1,1632423279
...,...,...,...,...
529308,3c4d0011-f473-44fa-9b65-e9118a51a955,2719,2,1624616329
539090,b5633cde-5f00-4e47-a1c4-01d2b4c56aff,2719,2,1635050924
550641,25670cd5-c9bf-4ec1-b532-eecdd293b812,2719,2,1635659599
551352,29396054-3374-471f-b4cd-b793750e5d41,2719,2,1636366687


### Recommendation using Implict

In [104]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler

In [111]:
dataset_df = dataset.copy()
dataset_df.rename(columns={'rating':'event_strength'}, inplace=True)

In [148]:
dataset_transformed = dataset_df.copy()
dataset_transformed['user'] = dataset_transformed['user'].astype("category")
dataset_transformed['user_id'] = dataset_transformed['user'].cat.codes

sparse_item_user = sparse.csr_matrix((dataset_transformed['event_strength'].astype(float), (dataset_transformed['item'], dataset_transformed['user_id'])))
sparse_user_item = sparse.csr_matrix((dataset_transformed['event_strength'].astype(float), (dataset_transformed['user_id'], dataset_transformed['item'])))


model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (sparse_item_user * alpha).astype('double')

model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [141]:
dataset_transformed.head()

,user,item,event_strength,timestamp,user_id
0,36c0daa2-5aad-4b0c-81e3-8d276b223573,6198,1,1633082142,4919
1,c9a9b40a-90ce-42c7-8b02-1301ad4914b1,190,1,1630125341,18023
2,83969882-083c-4150-b9eb-19bb3197720d,7330,1,1630143229,11813
3,83969882-083c-4150-b9eb-19bb3197720d,8978,1,1630143685,11813
4,83969882-083c-4150-b9eb-19bb3197720d,9263,1,1630143781,11813


#### Similar Items

In [138]:
item_id = 2719
n_similar = 10

user_vecs = model.user_factors
item_vecs = model.item_factors

item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

scores = item_vecs.dot(item_vecs[item_id]) / item_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

for item in similar:
    idx, score = item
    print(idx, score)

2719 1.0000001
5720 0.87037086
15838 0.8476719
22584 0.84492004
9098 0.84096164
5885 0.8357961
8698 0.83492094
2176 0.8272938
22466 0.8253766
14947 0.821198


In [170]:
def recommend(user_id, sparse_user_item, user_vecs, item_vecs, num_contents=10):
    user_interactions = sparse_user_item[user_id,:].toarray()
    user_interactions = user_interactions.reshape(-1) + 1
    user_interactions[user_interactions > 1] = 0
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    items = []
    scores = []
    
    for idx in content_idx:
        items.append(idx)
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'item': items, 'score': scores})

    return recommendations

user_vecs = sparse.csr_matrix(model.user_factors)
item_vecs = sparse.csr_matrix(model.item_factors)

user_id = 18023

recommendations = recommend(user_id, sparse_item_user, user_vecs, item_vecs)

print(recommendations)

    item     score
0  16534  1.000000
1   7116  0.959908
2    169  0.950384
3  21447  0.928597
4  22323  0.910530
5    325  0.898174
6   6341  0.897446
7  14359  0.894313
8  15797  0.890978
9  21660  0.876455


### LightFm Modeling

In [353]:
lf_dataset_df = dataset_df.copy()
new_products = pd.read_csv('data/catalogue.csv')
new_products_df = new_products.copy()
new_products_df.head()

,id,title,description,availability,condition,price,sale_price,link,brand,image_link
0,14521,"Oxy 10, 10gm","Oxy 10, 10gm",out of stock,new,318.00 NPR,286.20 NPR,https://jeevee.com/products?value=14521&type=d...,OXY,https://jeeveedev.s3.us-east-2.amazonaws.com/w...
1,12352,Make-Up Studio Brush Cleanser(Ph0900),Make-Up Studio Brush Cleanser(Ph0900),in stock,new,1785.00 NPR,1785.00 NPR,https://jeevee.com/products?value=12352&type=d...,MAKE-UP STUDIO,https://jeeveedev.s3.us-east-2.amazonaws.com/w...
2,11507,Buddsbuddy Silicone Orthodontic Pacifier (1pc),Buddsbuddy Silicone Orthodontic Pacifier (1pc),in stock,new,199.00 NPR,169.15 NPR,https://jeevee.com/products?value=11507&type=d...,BUDDSBUDDY,https://jeeveedev.s3.us-east-2.amazonaws.com/w...
3,12401,Make-Up Studio Cream Blusher,Colour-Rebellious Red,in stock,new,2200.00 NPR,2200.00 NPR,https://jeevee.com/products?value=12401&type=d...,MAKE-UP STUDIO,https://jeeveedev.s3.us-east-2.amazonaws.com/w...
4,14408,Set Wet Hair Gel Cool- 50 Ml,Set Wet Hair Gel Cool- 50 Ml,in stock,new,104.00 NPR,104.00 NPR,https://jeevee.com/products?value=14408&type=d...,SET WET,https://jeeveedev.s3.us-east-2.amazonaws.com/w...


In [354]:
new_products_df.rename(columns={"id":"item_id"}, inplace=True)
new_products_df = new_products_df[['item_id', 'title', 'price','brand']]
new_products_df['price'] = new_products_df['price'].str.replace(" NPR", '')
new_products_df['price'] = new_products_df['price'].str.strip()
new_products_df['price'] = new_products_df['price'].astype(float)
# new_products_df['price'] = new_products_df['price'].astype(int)
new_products_df.head()

,item_id,title,price,brand
0,14521,"Oxy 10, 10gm",318.0,OXY
1,12352,Make-Up Studio Brush Cleanser(Ph0900),1785.0,MAKE-UP STUDIO
2,11507,Buddsbuddy Silicone Orthodontic Pacifier (1pc),199.0,BUDDSBUDDY
3,12401,Make-Up Studio Cream Blusher,2200.0,MAKE-UP STUDIO
4,14408,Set Wet Hair Gel Cool- 50 Ml,104.0,SET WET


In [355]:
new_products_df[(new_products_df['item_id'] == 14534)]

,item_id,title,price,brand


In [409]:
items = new_products_df.copy()
users = pd.read_csv('data/users.csv')

users_df = users.copy()
items_df = items.copy()


users_df.dropna(inplace=True)
items_df.dropna(inplace=True)

# items_df = items_df.append({'item_id':14534, "brand":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':8467, "brand":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':14780, "brand":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':14781, "brand":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':14793, "brand":"SKIN CARE", "price":3034}, ignore_index=True)

# items_df = items_df.append({'item_id':6198, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':190, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':14313, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':13480, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':9365, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':10618, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':10605, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':2772, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df = items_df.append({'item_id':14364, "category":"SKIN CARE", "price":3034}, ignore_index=True)
# items_df['item_id'].append(9263)
# items_df['category'].append("SKIN CARE")
# items_df['price'].append(3043)

# 
# items_df['item_id'].append(6198)
# items_df['category'].append("SKIN CARE")
# items_df['price'].append(3043)

# items_df['item_id'].append(190)
# items_df['category'].append("SKIN CARE")
# items_df['price'].append(3043)


dataset_df[(dataset_df['item'] == 14534)]

# items_df[(items_df['item_id'] == 12356)]

,user,item,event_strength,timestamp
39,1fa52161-8421-411f-9f18-6df6eeb31a40,14534,1,1630388273
16514,292d9b11-f606-4587-bb86-0d75a25b4a7a,14534,1,1631168632
47768,baaa2d0b-2b42-4a36-b003-3464b9e89ad4,14534,1,1633912402
49205,792a6084-acee-430f-b40d-81254107ca6f,14534,1,1630524125
49252,792a6084-acee-430f-b40d-81254107ca6f,14534,1,1630610737
106707,7823865d-abe5-4644-ac40-07192fde9cd6,14534,1,1630389679
106924,7823865d-abe5-4644-ac40-07192fde9cd6,14534,1,1631317301
110312,543f2a07-6853-4ffc-8bfd-a5c590c5decc,14534,1,1630618940
139236,9df6c9a3-c48c-41b5-aa85-f5ea8f021242,14534,1,1633205519
275322,04ce8a07-54af-4ffe-98c4-cda8de2f22f6,14534,1,1630361888


In [425]:
b = items_df['item_id'].sort_values().values
a = sorted(dataset_df['item'].unique())
data = list(set(a) - set(b))
for value in data:
    lf_dataset_df.drop(lf_dataset_df.index[lf_dataset_df['item'] == value], inplace = True)

In [389]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(users_df['id'].values, items_df['item_id'].values)

In [250]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 47277, num_items 6640.


In [371]:
notYetIncluded = lf_dataset_df[~lf_dataset_df["user"].astype(str).isin(users_df["id"].astype(str))]

for index, row in notYetIncluded.iterrows():
    lf_dataset_df.drop(lf_dataset_df.index[lf_dataset_df['item'] == row['item']], inplace = True)

In [374]:
lf_dataset_df.to_csv('data/refined_user_item_interactions.csv', index=False)

In [372]:
interactions, weights = dataset.build_interactions((row['user'], row['item'], row['event_strength']) for index, row in lf_dataset_df.iterrows())


In [376]:
items_df

,item_id,title,price,brand
0,14521,"Oxy 10, 10gm",318.0,OXY
1,12352,Make-Up Studio Brush Cleanser(Ph0900),1785.0,MAKE-UP STUDIO
2,11507,Buddsbuddy Silicone Orthodontic Pacifier (1pc),199.0,BUDDSBUDDY
3,12401,Make-Up Studio Cream Blusher,2200.0,MAKE-UP STUDIO
4,14408,Set Wet Hair Gel Cool- 50 Ml,104.0,SET WET
...,...,...,...,...
14750,14534,NaN,3034.0,SKIN CARE
14751,8467,NaN,3034.0,SKIN CARE
14752,14780,NaN,3034.0,SKIN CARE
14753,14781,NaN,3034.0,SKIN CARE


In [410]:
items_df['brand'] = items_df['brand'].astype('category')
items_df['brand'] = items_df['brand'].cat.codes
items_features = [(row['price'], row['brand']) for index, row in items_df.iterrows()]
items_df.head()

,item_id,title,price,brand
0,14521,"Oxy 10, 10gm",318.0,321
1,12352,Make-Up Studio Brush Cleanser(Ph0900),1785.0,264
2,11507,Buddsbuddy Silicone Orthodontic Pacifier (1pc),199.0,57
3,12401,Make-Up Studio Cream Blusher,2200.0,264
4,14408,Set Wet Hair Gel Cool- 50 Ml,104.0,381


In [550]:
items_df.to_csv("data/processed_products.csv", index=False)

In [412]:
users_df.head()

,id,gender,date_of_birth
0,179db347-97e2-4e93-97c8-e0dba4dc69ae,MALE,1983-08-03
2,d023a4f1-5b6b-4d5d-9f7a-5d7b9a4d06d4,MALE,1982-12-20
3,f157ba08-cc44-4bb7-b35e-6ab1bfd19944,MALE,1991-05-30
5,802cff98-87be-4a92-bc4e-9064241b1707,MALE,1995-07-20
6,25a3bca3-d978-4962-8a24-da295535b2aa,FEMALE,2006-01-01


In [422]:
from datetime import datetime, date
def calculate_age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

users_df['date_of_birth'] = users_df['date_of_birth'].astype('str')
users_df['age'] = users_df['date_of_birth'].apply(calculate_age)
users_df['gender'] = users_df['gender'].astype('category')
users_df['gender'] = users_df['gender'].cat.codes
user_features =  [(row['age'], row['gender']) for index, row in users_df.iterrows()]

In [481]:
dataset.fit(users_df['id'].values, items_df['item_id'].values, item_features=items_features,user_features=user_features)

In [482]:
interactions, weights = dataset.build_interactions((row['user'], row['item'], row['event_strength']) for index, row in lf_dataset_df.iterrows())

In [439]:
type(interactions)

scipy.sparse.coo.coo_matrix

In [497]:
from lightfm import LightFM

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(interactions, epochs=1000)

In [508]:
def get_similar_product(model, item_id):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors
    
    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings.T
                      / np.linalg.norm(model.item_embeddings, axis=1)).T
    
    query_embedding = tag_embeddings[item_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:10]
    
    return most_similar

print(items_df[(items_df['item_id']==14408)])

similar_products = get_similar_product(model,14408)
for product in similar_products:
    print(items_df['title'][product])

   item_id                         title  price  brand
4    14408  Set Wet Hair Gel Cool- 50 Ml  104.0    381
Beardo Boar Brush
Fa Roll On Deodorant - Energy Boost, 50ml
Reizvoll Kissproof Lipistick
Note Luminous Moisturizing Foundation, 35ml Tube
Grace Deodorant Powder
Slurrp Farm Jaggery Powder, 250gm
Nacfil 600mg Tab
Golden Rose Hd Concealer High Definition
Lakme 9 To 5 Complexion Care Face Cc Cream Spf 30, 30 G


In [517]:
scores = model.predict(1000, np.arange(10))

In [518]:
np.argsort(-scores)

array([4, 3, 8, 5, 9, 7, 2, 0, 6, 1])

In [519]:
scores = model.predict(300, np.arange(10))
np.argsort(-scores)

array([4, 8, 3, 9, 0, 7, 5, 2, 1, 6])

In [535]:
scores = model.predict(1, np.arange(12000))
np.argsort(-scores)

array([ 9211, 10497,  1716, ..., 10011,  8513,  7026])

In [537]:
user_biases = model.get_user_representations()[0]
user_embeddings = model.get_user_representations()[1]
item_biases = model.get_item_representations()[0]
item_embeddings = model.get_item_representations()[1]
user1 = user_embeddings[1]
item1 = item_embeddings[1]
score = user1.dot(item1.T)
ub1 = user_biases[1]
ib1 = item_biases[1]

result = score + ub1 + ib1
predict = model.predict(1, np.arange(12000))
print(result)
print(np.argsort(-predict))

-0.33130473
[ 9211 10497  1716 ... 10011  8513  7026]


In [521]:
scores = model.predict(20000, np.arange(10))
items = np.argsort(-scores)
for item in items:
    print(items_df['title'][item])

Set Wet Hair Gel Cool- 50 Ml
Make-Up Studio Cream Blusher
The Moms Co. Natural Clay Deep Purifying Face Kit 
Maange 10 Pcs Professional Makeup Brushes With Bag
Earth Rhythm Neem Comb
Buddsbuddy Silicone Orthodontic Pacifier (1pc)
Oxy 10, 10gm
Make-Up Studio Eye Collection
Amayra Naturals Hemp & Rosemary Face Wash, 100ml
Make-Up Studio Brush Cleanser(Ph0900)


In [548]:
user_mapping = dataset.mapping()[0]
item_mapping = dataset.mapping()[2]
item_mapping

{14521: 0,
 12352: 1,
 11507: 2,
 12401: 3,
 14408: 4,
 14241: 5,
 7581: 6,
 5812: 7,
 12429: 8,
 12847: 9,
 4027: 10,
 12251: 11,
 14238: 12,
 15690: 13,
 16798: 14,
 7206: 15,
 13280: 16,
 7212: 17,
 8325: 18,
 11251: 19,
 12249: 20,
 8310: 21,
 18242: 22,
 5818: 23,
 7058: 24,
 14520: 25,
 15337: 26,
 16797: 27,
 804: 28,
 12495: 29,
 15322: 30,
 17343: 31,
 1351: 32,
 1749: 33,
 13259: 34,
 13260: 35,
 14246: 36,
 8599: 37,
 13261: 38,
 2983: 39,
 13262: 40,
 14523: 41,
 14117: 42,
 856: 43,
 17967: 44,
 9511: 45,
 16195: 46,
 15902: 47,
 7890: 48,
 14229: 49,
 15899: 50,
 13607: 51,
 13608: 52,
 15696: 53,
 17964: 54,
 15698: 55,
 13609: 56,
 17965: 57,
 13618: 58,
 13619: 59,
 13685: 60,
 13631: 61,
 773: 62,
 14118: 63,
 12933: 64,
 240: 65,
 17742: 66,
 12934: 67,
 19671: 68,
 15325: 69,
 14524: 70,
 14230: 71,
 15328: 72,
 15329: 73,
 73: 74,
 15703: 75,
 15323: 76,
 18243: 77,
 14859: 78,
 14860: 79,
 14861: 80,
 4641: 81,
 15327: 82,
 13634: 83,
 13686: 84,
 19035: 85,
 1320